In [180]:
import numpy as np

In [181]:

def discrete_simulated_bifurcation(ising_model, max_iterations=1000, alpha=0.5, beta=0.1, 
                                  decay_rate=0.999, threshold=1e-6, random_seed=None):
    """
    Implements the Discrete Simulated Bifurcation algorithm for optimizing Ising models.
    
    Parameters:
    -----------
    ising_model : object
        An object representing the Ising model. Must have the following methods:
        - get_coupling_matrix(): Returns the coupling matrix J
        - get_external_field(): Returns the external field vector h (optional)
        - get_size(): Returns the number of spins in the system
    max_iterations : int
        Maximum number of iterations to run the algorithm
    alpha : float
        Momentum coefficient (controls how much momentum is preserved)
    beta : float
        Driving force coefficient (controls the strength of the bifurcation driving force)
    decay_rate : float
        Rate at which beta decreases over iterations
    threshold : float
        Convergence threshold for spin changes
    random_seed : int or None
        Seed for random number generator
        
    Returns:
    --------
    spins : numpy.ndarray
        Final spin configuration (values are either +1 or -1)
    energy : float
        Energy of the final configuration
    iterations : int
        Number of iterations performed
    """
    # Set random seed if provided
    if random_seed is not None:
        np.random.seed(random_seed)
    
    # Get problem parameters from the Ising model
    n_spins = ising_model.get_size() # is the number of nodes in adjacency matrix/ rows in J
    J = ising_model.get_coupling_matrix() # is the adjacency matrix
    
    # Check if the model has an external field
    try:
        h = ising_model.get_external_field()
    except (AttributeError, NotImplementedError):
        h = np.zeros(n_spins) # h is zero
    
    # Initialize continuous spin variables (x) and their momentum (p)
    # Start with random values close to zero
    x = np.random.uniform(-0.1, 0.1, n_spins) # random small spins in shape of z
    p = np.zeros(n_spins) # start with 0 momentum p
    
    # Current beta value
    current_beta = beta # beta is the driving force
    
    # Track previous spin configuration for convergence check
    prev_spins = np.sign(x) # checking the spins of the randomly initiated spins
    
    # Main DSB loop
    for iteration in range(max_iterations):
        # Calculate the effective field from the coupling matrix and external field
        effective_field = h - J.dot(x) # h is zero, so dot product of spins and the adjacency matri/ coupling
        
        # Update momentum using the effective field
        p = alpha * p + effective_field # momentum = coeff(- dot product of spin and coupling)
        
        # Update continuous spin variables with momentum and nonlinear driving force
        # The tanh term creates the bifurcation effect, pushing spins toward +1 or -1
        driving_force = current_beta * x * (1.0 - x * x) # driving force = driving force * spin * (1 - spin * spin)
        x = x + p + driving_force # spin = spin + momentum + driving force
        
        # Apply soft constraints to keep x values within reasonable bounds
        x = np.clip(x, -1.0, 1.0) # clipping the spin between -1 and 1
        
        # Get the current discrete spin configuration
        current_spins = np.sign(x) # getting signs of spins
        
        # Check for convergence
        if np.sum(np.abs(current_spins - prev_spins)) / n_spins < threshold:
            # Calculate final energy
            energy = -0.5 * np.dot(current_spins, J.dot(current_spins)) - np.dot(h, current_spins)
            # if it converges, return the spins and energy
            return current_spins, energy, iteration + 1
        
        # Update previous spins
        # If it does not converge, update the previous spin
        prev_spins = current_spins.copy()
        
        # Decay beta (annealing)
        # if it does not converge, decay the driving force by the decay rate
        current_beta *= decay_rate
    
    # The loop continues till the convergence criterion is met
    # then final energy is calculated
    # and final spins are calculated
    final_spins = np.sign(x)
    final_energy = -0.5 * np.dot(final_spins, J.dot(final_spins)) - np.dot(h, final_spins)
    return final_spins, final_energy, max_iterations


# Example usage with a simple Ising model class
class SimpleIsingModel:
    """
    A simple implementation of an Ising model with a coupling matrix and external field.
    """
    def __init__(self, coupling_matrix, external_field=None):
        """
        Initialize the Ising model.
        
        Parameters:
        -----------
        coupling_matrix : numpy.ndarray
            Square matrix representing coupling strengths between spins
        external_field : numpy.ndarray or None
            Vector representing external field at each spin site
        """
        self.J = coupling_matrix
        self.n_spins = coupling_matrix.shape[0]
        
        if external_field is None:
            self.h = np.zeros(self.n_spins)
        else:
            self.h = external_field
    
    def get_coupling_matrix(self):
        """Return the coupling matrix J"""
        return self.J
    
    def get_external_field(self):
        """Return the external field vector h"""
        return self.h
    
    def get_size(self):
        """Return the number of spins in the system"""
        return self.n_spins
    
    def calculate_energy(self, spins):
        """
        Calculate the energy of a given spin configuration.
        
        Parameters:
        -----------
        spins : numpy.ndarray
            Spin configuration (values should be +1 or -1)
            
        Returns:
        --------
        energy : float
            Energy of the configuration
        """
        return -0.5 * np.dot(spins, self.J.dot(spins)) - np.dot(self.h, spins)


# Example of how to use the algorithm
if __name__ == "__main__":
    # Create a small random Ising problem (10 spins)
    n = 10
    np.random.seed(42)
    
    # Create a symmetric coupling matrix
    J_matrix = np.random.uniform(-1, 1, (n, n))
    J_matrix = (J_matrix + J_matrix.T) / 2  # Make it symmetric
    np.fill_diagonal(J_matrix, 0)  # No self-interactions
    
    # Create a random external field
    h_vector = np.random.uniform(-0.5, 0.5, n)
    
    # Create the Ising model
    ising_problem = SimpleIsingModel(J_matrix, h_vector)
    
    # Run the DSB algorithm
    final_spins, final_energy, iterations = discrete_simulated_bifurcation(
        ising_problem, max_iterations=500, alpha=0.5, beta=0.1, decay_rate=0.999
    )
    
    print(f"Optimization completed in {iterations} iterations")
    print(f"Final energy: {final_energy}")
    print(f"Final spin configuration: {final_spins}")

Optimization completed in 4 iterations
Final energy: 1.4997453541704497
Final spin configuration: [-1.  1.  1.  1.  1. -1. -1.  1. -1. -1.]


# Running the code for maxcut

In [182]:
# Example of how to use the algorithm
if __name__ == "__main__":
    # Create a small random Ising problem (10 spins)
    n = 10
    np.random.seed(42)
    
    # Create a symmetric coupling matrix
    # J_matrix = np.random.uniform(-1, 1, (n, n))
    # J_matrix = (J_matrix + J_matrix.T) / 2  # Make it symmetric
    # np.fill_diagonal(J_matrix, 0)  # No self-interactions

    J_matrix = np.array([
        [0, 1, 2, 1, 0, 3, 0],
        [1, 0, 1, 0, 2, 0, 3],
        [2, 1, 0, 3, 1, 0, 2],
        [1, 0, 3, 0, 1, 2, 0],
        [0, 2, 1, 1, 0, 3, 1],
        [3, 0, 0, 2, 3, 0, 1],
        [0, 3, 2, 0, 1, 1, 0]
    ])
    
    # Create a random external field
    # h_vector = np.random.uniform(-0.5, 0.5, n)
    h_vector = None
    
    # Create the Ising model
    ising_problem = SimpleIsingModel(J_matrix, h_vector)
    
    # Run the DSB algorithm
    final_spins, final_energy, iterations = discrete_simulated_bifurcation(
        ising_problem, max_iterations=500, alpha=0.5, beta=0.1, decay_rate=0.999
    )
    
    print(f"Optimization completed in {iterations} iterations")
    print(f"Final energy: {final_energy}")
    print(f"Final spin configuration: {final_spins}")

Optimization completed in 3 iterations
Final energy: 19.0
Final spin configuration: [-1.  1.  1. -1. -1.  1. -1.]
